In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=2a3dbe843096e878f97449039dc7cdd77cac390b30429b9c637c6721f1d88d37
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pandas as pd

from pyspark.sql import SparkSession

from pyspark.sql import *


In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
!git clone https://github.com/kdaga81/IPL-Auction.git

Cloning into 'IPL-Auction'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 9.90 KiB | 2.48 MiB/s, done.


In [8]:
from pyspark.sql.types import StructType, IntegerType, StringType, FloatType


In [9]:
schema = StructType() \
        .add("c0_",IntegerType(),True) \
        .add("Player", StringType(),True) \
        .add("Base Price",StringType(), True) \
        .add("TYPE",StringType(),True) \
        .add("COST IN ₹ (CR.)",FloatType(),True) \
        .add("Cost IN $ (000)",FloatType(),True) \
        .add("2021 Squad", StringType(),True) \
        .add("Team", StringType(),True)

In [10]:
df = spark.read.option("header", True).option('inferSchema',True).csv("/content/IPL-Auction/ipl_2022_dataset.csv")

In [11]:
df.show(5)

+---+---------------+----------+-----------+---------------+---------------+----------+--------------+
|_c0|         Player|Base Price|       TYPE|COST IN ₹ (CR.)|Cost IN $ (000)|2021 Squad|          Team|
+---+---------------+----------+-----------+---------------+---------------+----------+--------------+
|  0|    Rashid Khan|Draft Pick|     BOWLER|           15.0|         1950.0|       SRH|Gujarat Titans|
|  1|  Hardik Pandya|Draft Pick|ALL-ROUNDER|           15.0|         1950.0|        MI|Gujarat Titans|
|  2|Lockie Ferguson|      2 Cr|     BOWLER|           10.0|         1300.0|       KKR|Gujarat Titans|
|  3|  Rahul Tewatia|   40 Lakh|ALL-ROUNDER|            9.0|         1170.0|        RR|Gujarat Titans|
|  4|   Shubman Gill|Draft Pick|     BATTER|            8.0|         1040.0|       KKR|Gujarat Titans|
+---+---------------+----------+-----------+---------------+---------------+----------+--------------+
only showing top 5 rows



In [12]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Base Price: string (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- COST IN ₹ (CR.): double (nullable = true)
 |-- Cost IN $ (000): double (nullable = true)
 |-- 2021 Squad: string (nullable = true)
 |-- Team: string (nullable = true)



In [13]:
print('Rows: ', df.count())

print('Columns:', len(df.columns))

Rows:  633
Columns: 8


In [14]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Base Price: string (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- COST IN ₹ (CR.): double (nullable = true)
 |-- Cost IN $ (000): double (nullable = true)
 |-- 2021 Squad: string (nullable = true)
 |-- Team: string (nullable = true)



In [15]:
df.select('Base Price').distinct().collect()

[Row(Base Price='20 Lakh'),
 Row(Base Price='2 Cr'),
 Row(Base Price='Draft Pick'),
 Row(Base Price='40 Lakh'),
 Row(Base Price='30 Lakh'),
 Row(Base Price='1 Cr'),
 Row(Base Price='50 Lakh'),
 Row(Base Price='1.5 Cr'),
 Row(Base Price='75 Lakh'),
 Row(Base Price='Retained')]

In [16]:
df.groupBy('Base Price').count().show()

+----------+-----+
|Base Price|count|
+----------+-----+
|   20 Lakh|  344|
|      2 Cr|   48|
|Draft Pick|    6|
|   40 Lakh|   16|
|   30 Lakh|    9|
|      1 Cr|   33|
|   50 Lakh|  104|
|    1.5 Cr|   20|
|   75 Lakh|   26|
|  Retained|   27|
+----------+-----+



In [17]:
df2 = df.withColumnRenamed("Player","player")\
      .withColumnRenamed("Base Price","base_price")\
      .withColumnRenamed("Type","type") \
      .withColumnRenamed("COST IN ₹ (CR.)","cost_inr") \
      .withColumnRenamed("Cost IN $ (000)","cost_usd") \
      .withColumnRenamed("2021 Squad","2021_team") \
      .withColumnRenamed("Team","2022_team")

In [19]:
df.show(5)

+---+---------------+----------+-----------+---------------+---------------+----------+--------------+
|_c0|         Player|Base Price|       TYPE|COST IN ₹ (CR.)|Cost IN $ (000)|2021 Squad|          Team|
+---+---------------+----------+-----------+---------------+---------------+----------+--------------+
|  0|    Rashid Khan|Draft Pick|     BOWLER|           15.0|         1950.0|       SRH|Gujarat Titans|
|  1|  Hardik Pandya|Draft Pick|ALL-ROUNDER|           15.0|         1950.0|        MI|Gujarat Titans|
|  2|Lockie Ferguson|      2 Cr|     BOWLER|           10.0|         1300.0|       KKR|Gujarat Titans|
|  3|  Rahul Tewatia|   40 Lakh|ALL-ROUNDER|            9.0|         1170.0|        RR|Gujarat Titans|
|  4|   Shubman Gill|Draft Pick|     BATTER|            8.0|         1040.0|       KKR|Gujarat Titans|
+---+---------------+----------+-----------+---------------+---------------+----------+--------------+
only showing top 5 rows



In [18]:
df2.show(5)

+---+---------------+----------+-----------+--------+--------+---------+--------------+
|_c0|         player|base_price|       type|cost_inr|cost_usd|2021_team|     2022_team|
+---+---------------+----------+-----------+--------+--------+---------+--------------+
|  0|    Rashid Khan|Draft Pick|     BOWLER|    15.0|  1950.0|      SRH|Gujarat Titans|
|  1|  Hardik Pandya|Draft Pick|ALL-ROUNDER|    15.0|  1950.0|       MI|Gujarat Titans|
|  2|Lockie Ferguson|      2 Cr|     BOWLER|    10.0|  1300.0|      KKR|Gujarat Titans|
|  3|  Rahul Tewatia|   40 Lakh|ALL-ROUNDER|     9.0|  1170.0|       RR|Gujarat Titans|
|  4|   Shubman Gill|Draft Pick|     BATTER|     8.0|  1040.0|      KKR|Gujarat Titans|
+---+---------------+----------+-----------+--------+--------+---------+--------------+
only showing top 5 rows



In [20]:
df3 = df2.drop('cost_usd')

In [21]:
df3.show()

+---+-------------------+----------+------------+--------+---------+--------------+
|_c0|             player|base_price|        type|cost_inr|2021_team|     2022_team|
+---+-------------------+----------+------------+--------+---------+--------------+
|  0|        Rashid Khan|Draft Pick|      BOWLER|    15.0|      SRH|Gujarat Titans|
|  1|      Hardik Pandya|Draft Pick| ALL-ROUNDER|    15.0|       MI|Gujarat Titans|
|  2|    Lockie Ferguson|      2 Cr|      BOWLER|    10.0|      KKR|Gujarat Titans|
|  3|      Rahul Tewatia|   40 Lakh| ALL-ROUNDER|     9.0|       RR|Gujarat Titans|
|  4|       Shubman Gill|Draft Pick|      BATTER|     8.0|      KKR|Gujarat Titans|
|  5|     Mohammad Shami|      2 Cr|      BOWLER|    6.25|     PBKS|Gujarat Titans|
|  6|         Yash Dayal|   20 Lakh|      BOWLER|     3.2|     NULL|Gujarat Titans|
|  7|       David Miller|      1 Cr|      BATTER|     3.0|       RR|Gujarat Titans|
|  8|     R. Sai Kishore|   20 Lakh|      BOWLER|     3.0|      CSK|Gujarat 

In [22]:
df3.select('player','cost_inr').where(df3.type=='BATTER').orderBy('cost_inr', ascending=False).show(3)

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|   Rohit Sharma|    16.0|
|    Virat Kohli|    15.0|
|Kane Williamson|    14.0|
+---------------+--------+
only showing top 3 rows



In [23]:
df3.createOrReplaceTempView('ipl')

In [24]:
spark.sql("""SELECT player, cost_inr FROM ipl WHERE type = 'BATTER' ORDER BY 2 DESC LIMIT 5""").show()

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|   Rohit Sharma|    16.0|
|    Virat Kohli|    15.0|
|Kane Williamson|    14.0|
|   Shreyas Iyer|   12.25|
| Mayank Agarwal|    12.0|
+---------------+--------+



In [25]:
df3.select('player','cost_inr').filter(df3.type=='BOWLER').orderBy('cost_inr',ascending=False).show(5)

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|    Rashid Khan|    15.0|
|  Deepak Chahar|    14.0|
| Jasprit Bumrah|    12.0|
| Shardul Thakur|   10.75|
|Lockie Ferguson|    10.0|
+---------------+--------+
only showing top 5 rows



In [26]:
spark.sql("""SELECT player, cost_inr FROM ipl WHERE type = 'BOWLER' ORDER By 2 DESC LIMIT 5 """).show()

+--------------+--------+
|        player|cost_inr|
+--------------+--------+
|   Rashid Khan|    15.0|
| Deepak Chahar|    14.0|
|Jasprit Bumrah|    12.0|
|Shardul Thakur|   10.75|
|    Avesh Khan|    10.0|
+--------------+--------+



In [27]:
df3.select('player', 'cost_inr').filter((df3.type=='WICKETKEEPER')&(df3.cost_inr.isNotNull())).orderBy('cost_inr',ascending=True).show(5)

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|  Jitesh Sharma|     0.2|
|    Aryan Juyal|     0.2|
|  N. Jagadeesan|     0.2|
|Luvnith Sisodia|     0.2|
| Baba Indrajith|     0.2|
+---------------+--------+
only showing top 5 rows



In [28]:
spark.sql(""" SELECT player, cost_inr FROM ipl WHERE type = 'WICKETKEEPER' AND cost_inr is not null ORDER BY 2 LIMIT 5 """).show()

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|    Dhruv Jurel|     0.2|
|  Jitesh Sharma|     0.2|
|  N. Jagadeesan|     0.2|
|    Aryan Juyal|     0.2|
|Luvnith Sisodia|     0.2|
+---------------+--------+



In [29]:
from pyspark.sql import functions as F

df3.groupBy('type').agg(F.round(F.mean('cost_inr'),2).alias('average_price')).show()

+------------+-------------+
|        type|average_price|
+------------+-------------+
|WICKETKEEPER|         5.09|
|      BOWLER|         3.07|
| ALL-ROUNDER|         3.61|
|      BATTER|         4.11|
+------------+-------------+



In [30]:
spark.sql("""SELECT type, round(avg(cost_inr),2) average_price FROM ipl GROUP BY 1 ORDER BY 2 DESC""").show()

+------------+-------------+
|        type|average_price|
+------------+-------------+
|WICKETKEEPER|         5.09|
|      BATTER|         4.11|
| ALL-ROUNDER|         3.61|
|      BOWLER|         3.07|
+------------+-------------+



In [32]:
df3.select('player','2021_team','cost_inr').where(df3.base_price=='Retained').orderBy('cost_inr',ascending=False).show(10)


+---------------+---------+--------+
|         player|2021_team|cost_inr|
+---------------+---------+--------+
|Ravindra Jadeja|      CSK|    16.0|
|   Rishabh Pant|       DC|    16.0|
|   Rohit Sharma|       MI|    16.0|
|    Virat Kohli|      RCB|    15.0|
|   Sanju Samson|       RR|    14.0|
|Kane Williamson|      SRH|    14.0|
|       MS Dhoni|      CSK|    12.0|
|  Andre Russell|      KKR|    12.0|
| Mayank Agarwal|     PBKS|    12.0|
| Jasprit Bumrah|       MI|    12.0|
+---------------+---------+--------+
only showing top 10 rows



In [33]:
spark.sql("""SELECT player, cost_inr FROM ipl WHERE base_price ="Retained" ORDER BY 2 DESC LIMIT 5 """).show()

+---------------+--------+
|         player|cost_inr|
+---------------+--------+
|Ravindra Jadeja|    16.0|
|   Rishabh Pant|    16.0|
|   Rohit Sharma|    16.0|
|    Virat Kohli|    15.0|
|   Sanju Samson|    14.0|
+---------------+--------+

